# Amazon Product Review Scraping

This project aims to scrape reviews of any product link provided using Python.

In [1]:
%%capture requirements
import sys
import os
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install pandas
import time
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
product_link = input('Enter the product link: ')

Enter the product link: https://www.amazon.de/dp/B07SJR6HL3?ref=dacx_dp_1486540050202_3693756170602&pd_rd_plhdr=t&aaxitk=oKhWNvWHnPkBHCjSwhrxXw


In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.3 (KHTML, like Gecko) Chrome/6.0.472.64 Safari/534.3'}
product_page = requests.get(product_link, headers=headers)
#print(product_page)

In [4]:
product_page_soup = BeautifulSoup(product_page.text, 'lxml')
#print(product_page_soup)

In [5]:
see_all_reviews = product_page_soup.find_all("a", {"data-hook" : "see-all-reviews-link-foot"})[0]
all_reviews_url =  "/".join(product_link.split("/", 3)[:3]) + see_all_reviews['href']
print(all_reviews_url)

https://www.amazon.de/Soundcore-Ger%C3%A4uschisolierung-kristallklares-Akkulaufzeit-Wasserschutzklasse-Schwarz/product-reviews/B07SJR6HL3/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [6]:
all_reviews_page = requests.get(all_reviews_url, headers=headers)
all_reviews_page_soup = BeautifulSoup(all_reviews_page.text, 'lxml')

#do
i=1
# review_id = [i['id'] for i in all_reviews_page_soup.find_all(class_='a-section review aok-relative')[2:]]
names = list([i.text for i in all_reviews_page_soup.find_all(class_='a-profile-name')[2:]])
ratings = list([i.find(class_='a-icon-alt').text[:3] for i in all_reviews_page_soup.find_all(class_='a-section celwidget')])
title = list([i.span.text for i in all_reviews_page_soup.find_all(class_='review-title')[2:]])
date = list([i.text for i in all_reviews_page_soup.find_all(class_='review-date')[2:]])
text = list([i.span.text for i in all_reviews_page_soup.find_all(class_='review-text-content')])
#while
# print(type(all_reviews_page_soup.find(class_='a-pagination').find_all('li')[1].a))
try:
    while all_reviews_page_soup.find(class_='a-pagination'):
        if all_reviews_page_soup.find(class_='a-pagination').find_all('li')[1].a:
            #print(i)
            i+=1
            next_url = 'https://www.amazon.in' + all_reviews_page_soup.find(class_='a-pagination').find_all('li')[1].a['href']
            all_reviews_page = requests.get(next_url)
            all_reviews_page_soup = BeautifulSoup(all_reviews_page.text, 'html.parser')
    #         review_id.append([i['id'] for i in all_reviews_page_soup.find_all(class_='a-section review aok-relative')[2:]])
            temp_names = [i.text for i in all_reviews_page_soup.find_all(class_='a-profile-name')[2:]]
            if temp_names:
                names.append(temp_names)
            else:
                names.append('')
            temp_ratings = [i.find(class_='a-icon-alt').text[:3] for i in all_reviews_page_soup.find_all(class_='a-section celwidget')]
            if temp_ratings:
                ratings.append(temp_ratings)
            else:
                ratings.append('')
            temp_title = [i.span.text for i in all_reviews_page_soup.find_all(class_='review-title')[2:]]
            if temp_title:
                title.append(temp_title)
            else:
                title.append('')
            temp_date = [i.text for i in all_reviews_page_soup.find_all(class_='review-date')[2:]]
            if temp_date:
                date.append(temp_date)
            else:
                date.append('')
            temp_text = [i.span.text for i in all_reviews_page_soup.find_all(class_='review-text-content')]
            if temp_text:
                text.append(temp_text)
            else:
                text.append('')
        else:
            break
except Exception as e:
    print(e, type(all_reviews_page_soup.find(class_='a-pagination')))
#print('Name'+str(len(names))+'Ratings'+str(len(ratings))+'Title'+str(len(title))+'Date'+str(len(date))+'Text'+str(len(text)))
reviews_dict = {'Name':names, 'Ratings':ratings, 'Title':title, 'Date':date, 'Text':text}
#print(reviews_dict)

In [7]:
reviews_df = pd.DataFrame.from_dict(reviews_dict)
#print(reviews_df)

Saving the the Reviews dataframe to a CSV file.

In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")
csvoutput_file = f'.{os.path.sep}review_{timestr}.csv'
#print(f'saving to {csvoutput_file}')
reviews_df.to_csv(csvoutput_file)